# TD2 — Opérations OLAP et Analyse Multidimensionnelle

**Durée estimée** : 1h30  
**Niveau** : Intermédiaire  
**Objectifs pédagogiques** :
1.  **Comprendre** la notion de Cube OLAP (Dimensions x Mesures).
2.  **Pratiquer** les opérations fondamentales : *Slice, Dice, Roll-up, Drill-down*.
3.  **Maîtriser** les fonctions analytiques SQL avancées : `RANK()`, `LAG()`, `GROUP BY`.
4.  **Vérifier** la cohérence des agrégations.

---

## Partie 1 : Préparation de l'environnement (Mini-Datawarehouse)

Pour réaliser ce TD, nous avons besoin d'un schéma en étoile peuplé.
Nous reprenons la structure du TD1 (Ventes Retail) et nous l'initialisons en mémoire.

*Exécutez cette cellule pour créer et charger les données.*

In [7]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 1. DDL (Structure)
cursor.executescript('''
CREATE TABLE dim_date (
  date_id INTEGER PRIMARY KEY, date_cal DATE, annee INTEGER, mois INTEGER, trimestre INTEGER
);
CREATE TABLE dim_produit (
  produit_sk INTEGER PRIMARY KEY, produit_nom TEXT, categorie TEXT
);
CREATE TABLE dim_magasin (
  magasin_sk INTEGER PRIMARY KEY, magasin_id TEXT, ville TEXT, region TEXT
);
CREATE TABLE fact_ventes (
  fact_sk INTEGER PRIMARY KEY, date_id INTEGER, produit_sk INTEGER, magasin_sk INTEGER, 
  montant REAL, quantite INTEGER
);
''')

# 2. Chargement des données (Jeu d'essai étendu pour le TD2)
cursor.executescript('''
-- Dimensions
INSERT INTO dim_date VALUES 
(1, '2024-01-02', 2024, 1, 1), (2, '2024-01-15', 2024, 1, 1),
(3, '2024-02-01', 2024, 2, 1), (4, '2024-02-20', 2024, 2, 1);

INSERT INTO dim_produit VALUES 
(1, 'Chemise Oxford', 'Textile'), (2, 'Sneakers Run', 'Chaussure'), 
(3, 'Jeans Slim', 'Textile'), (4, 'Montre Sport', 'Accessoire');

INSERT INTO dim_magasin VALUES 
(1, 'M01', 'Paris', 'IDF'), (2, 'M02', 'Lyon', 'ARA'), (3, 'M03', 'Lille', 'HDF');

-- Faits (Ventes)
INSERT INTO fact_ventes (date_id, produit_sk, magasin_sk, montant, quantite) VALUES 
(1, 1, 1, 120.0, 2), (1, 2, 1, 75.0, 1),  -- Janvier Paris
(2, 3, 1, 90.0, 1), (2, 4, 1, 200.0, 1), -- Janvier Paris
(3, 1, 2, 60.0, 1), (3, 2, 2, 75.0, 1),  -- Février Lyon
(4, 3, 3, 90.0, 1), (4, 1, 3, 60.0, 1);  -- Février Lille
''')

print('✅ Environnement OLAP prêt (Base en mémoire).')

✅ Environnement OLAP prêt (Base en mémoire).


## Partie 2 : Concepts et Opérations Fondamentales

Nous travaillons sur un **Cube** défini par 3 axes : **Temps**, **Produit**, **Géographie**.

### Représentation Conceptuelle des Opérations
```mermaid
graph TD
    subgraph "Cube Initial"
    C1[Ventes Globales]
    end
    
    subgraph "Opérations"
    C1 -->|Drill-down| O1[Vétail par Mois]
    C1 -->|Slice| O2[Ventes Janvier uniquement]
    C1 -->|Dice| O3[Ventes Janvier & Textile]
    O1 -->|Roll-up| O4[Total Annuel]
    end
    
    style C1 fill:#f9f,stroke:#333
```

### 1. Roll-up (Agrégation)
**Définition** : Remonter dans la hiérarchie (ex: Jour -> Mois -> Année) ou supprimer une dimension d'analyse.
*Exemple : Quel est le CA mensuel par Magasin ? (On oublie le détail Produit)*

In [8]:
# Opération ROLL-UP : Somme par Mois et par Magasin
sql_rollup = '''
SELECT 
    d.mois,
    m.ville,
    SUM(f.montant) as ca_mensuel
FROM fact_ventes f
JOIN dim_date d ON f.date_id = d.date_id
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
GROUP BY d.mois, m.ville
ORDER BY d.mois, m.ville;
'''
print("📊 Résultat Roll-up (Mois x Ville) :")
display(pd.read_sql_query(sql_rollup, conn))

📊 Résultat Roll-up (Mois x Ville) :


,mois,ville,ca_mensuel
0,1,Paris,485.0
1,2,Lille,150.0
2,2,Lyon,135.0


### 2. Slice & Dice (Filtrage)
- **Slice** : Filtrer sur une valeur unique d'une dimension (une "tranche").
- **Dice** : Filtrer sur un sous-ensemble de plusieurs dimensions (un "dé").

*Exemple Dice : Ventes de la catégorie 'Textile' en région 'IDF'.*

In [9]:
# Opération DICE : Textile en IDF
sql_dice = '''
SELECT 
    p.produit_nom,
    m.ville,
    f.montant
FROM fact_ventes f
JOIN dim_produit p ON f.produit_sk = p.produit_sk
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
WHERE p.categorie = 'Textile'  -- Axe 1
  AND m.region = 'IDF';        -- Axe 2
'''
print("📊 Résultat Dice (Textile + IDF) :")
display(pd.read_sql_query(sql_dice, conn))

📊 Résultat Dice (Textile + IDF) :


,produit_nom,ville,montant
0,Chemise Oxford,Paris,120.0
1,Jeans Slim,Paris,90.0


In [10]:
### 3. Drill-down (Désagrégation)
**Définition** : Descendre dans la hiérarchie pour voir le détail (ex: Année -> Mois).
*Exemple : On a vu le CA par ville. Maintenant, zoomons sur 'Paris' pour voir le détail par Produit.*

SyntaxError: invalid syntax (1842907361.py, line 2)

In [ ]:
# Opération DRILL-DOWN : Détail Paris
sql_drilldown = '''
SELECT 
    d.mois,
    p.produit_nom,
    SUM(f.montant) as ca_mensuel
FROM fact_ventes f
JOIN dim_date d ON f.date_id = d.date_id
JOIN dim_produit p ON f.produit_sk = p.produit_sk
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
WHERE m.ville = 'Paris'  -- Focus sur une branche
GROUP BY d.mois, p.produit_nom
ORDER BY d.mois, ca_mensuel DESC;
'''
print("📊 Résultat Drill-down (Paris -> Produits) :")
display(pd.read_sql_query(sql_drilldown, conn))

In [ ]:
## Partie 3 : Analyse Avancée (Window Functions)

SQL permet des analyses complexes sans passer par du code Python, grâce aux **Fonctions de Fenêtrage (Window Functions)**.

### 1. Classement (Ranking)
Identifier les meilleurs produits par mois sans réduire le nombre de lignes.
Utilisation de `RANK()` ou `DENSE_RANK()`.

In [ ]:
# Classement des produits par CA pour chaque Mois
sql_rank = '''
SELECT 
    d.mois,
    p.produit_nom,
    SUM(f.montant) as ca,
    RANK() OVER (
        PARTITION BY d.mois 
        ORDER BY SUM(f.montant) DESC
    ) as rang
FROM fact_ventes f
JOIN dim_date d ON f.date_id = d.date_id
JOIN dim_produit p ON f.produit_sk = p.produit_sk
GROUP BY d.mois, p.produit_nom
ORDER BY d.mois, rang;
'''
print("📊 Résultat Ranking (Top produits par mois) :")
display(pd.read_sql_query(sql_rank, conn))

In [ ]:
### 2. Analyse Comparative (Time Intelligence)
Comparer une période avec la précédente (Year-Over-Year, Month-Over-Month).
Utilisation de `LAG()` pour accéder à la ligne précédente.

In [ ]:
# Evolution du CA Mensuel (Mois courant vs Mois précédent)
sql_lag = '''
WITH ca_mensuel AS (
    SELECT 
        d.mois,
        SUM(f.montant) as ca
    FROM fact_ventes f
    JOIN dim_date d ON f.date_id = d.date_id
    GROUP BY d.mois
)
SELECT 
    mois,
    ca,
    LAG(ca, 1, 0) OVER (ORDER BY mois) as ca_mois_precedent,
    (ca - LAG(ca, 1, 0) OVER (ORDER BY mois)) as evolution_abs,
    ROUND((ca - LAG(ca, 1, 0) OVER (ORDER BY mois)) / LAG(ca, 1, 0) OVER (ORDER BY mois) * 100, 2) as evolution_pct
FROM ca_mensuel;
'''
print("📊 Résultat LAG (Croissance mensuelle) :")
display(pd.read_sql_query(sql_lag, conn))

In [ ]:
### 3. Pivot (Tableau Croisé)
Transformer des lignes en colonnes pour une meilleure lisibilité.
Exemple : Produits en lignes, Magasins en colonnes.

In [ ]:
# Pivot : CA par Produit (Lignes) et Magasin (Colonnes)
# En SQL standard, on utilise des CASE WHEN conditionnels avec une agrégation.

sql_pivot = '''
SELECT 
    p.produit_nom,
    SUM(CASE WHEN m.ville = 'Paris' THEN f.montant ELSE 0 END) as ca_paris,
    SUM(CASE WHEN m.ville = 'Lyon' THEN f.montant ELSE 0 END) as ca_lyon,
    SUM(CASE WHEN m.ville = 'Lille' THEN f.montant ELSE 0 END) as ca_lille,
    SUM(f.montant) as total_produit
FROM fact_ventes f
JOIN dim_produit p ON f.produit_sk = p.produit_sk
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
GROUP BY p.produit_nom
ORDER BY total_produit DESC;
'''
print("📊 Résultat Pivot (Produits x Magasins) :")
display(pd.read_sql_query(sql_pivot, conn))

In [ ]:
## Partie 4 : Contrôles de Qualité des Données

Dans un projet BI, la confiance est clé. Il faut vérifier que les agrégations sont cohérentes.

**Règles à vérifier :**
1.  **Additivité** : La somme des détails doit être égale au total global.
2.  **Complétude** : Toutes les ventes doivent être rattachées à un magasin et un produit.

In [ ]:
# Test de cohérence : Total Annuel calculé de 2 manières

# 1. Total direct
cursor.execute("SELECT SUM(montant) FROM fact_ventes")
total_direct = cursor.fetchone()[0]

# 2. Somme des CA par magasin (Agrégat)
cursor.execute('''
SELECT SUM(ca_mag) FROM (
    SELECT magasin_sk, SUM(montant) as ca_mag 
    FROM fact_ventes 
    GROUP BY magasin_sk
)
''')
total_agreg = cursor.fetchone()[0]

print(f"💰 Total Direct : {total_direct} €")
print(f"∑  Total Agrégé : {total_agreg} €")

if abs(total_direct - total_agreg) < 0.01:
    print("✅ Le cube est cohérent (Additivité respectée).")
else:
    print("❌ ALERTE : Incohérence détectée !")

conn.close()
print('\n✅ TD2 terminé avec succès.')